In [1]:
from models import ChatMessage, ChatHistory
import dspy
from lms.together import Together

from modules.chatter import ChatterModule

C:\Users\subho\anaconda3\envs\whisper2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lm = Together(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
    temperature=0.5,
    max_tokens=1000,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1.2,
    stop=["<|eot_id|>", "<|eom_id|>", "\n\n---\n\n", "\n\n---", "---", "\n---"],
    # stop=["\n", "\n\n"],
)

In [3]:
dspy.settings.configure(lm=lm)

In [4]:
import json
import random
from models import ChatMessage, ChatHistory

# Load the dataset
with open('../training_data/conversations.json', 'r') as f:
    conversations = json.load(f)

# Set random seed for reproducibility
random.seed(42)
random.shuffle(conversations)

# Split into train (80%) and dev (20%) sets
split_idx = int(len(conversations) * 0.8)
train_data = conversations[:split_idx]
dev_data = conversations[split_idx:]

# Create trainset and devset with proper ChatHistory objects
def create_example(conv):
    # Create ChatHistory object with proper message format
    chat_history = ChatHistory(
        messages=[
            ChatMessage(**msg) for msg in conv['chat_history']['messages']
        ]
    )
    
    return dspy.Example({
        'chat_history': chat_history,
        'output': conv['output']
    }).with_inputs('chat_history')

trainset = [create_example(conv) for conv in train_data]
devset = [create_example(conv) for conv in dev_data]

print(f"Training examples: {len(trainset)}")
print(f"Dev examples: {len(devset)}")

Training examples: 8
Dev examples: 2


In [5]:
# Example from trainset
example = trainset[0]
print("Training Example Structure:")
print(example.chat_history)  # This is a ChatHistory object
print(example.chat_history.messages) 

Training Example Structure:
THE FAN: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
YOU: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
THE FAN: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
[ChatMessage(from_creator=False, content='I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?'), ChatMessage(from_creator=True, content='Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.'), ChatMessage(from_creator=False, content='It feels chaotic, but I also feel like Iâ€™m learning so much about myself.')]


In [14]:
from dspy.teleprompt import KNNFewShot
from modules.chatter import ChatterModule

# Create the KNN teleprompter
knn_teleprompter = KNNFewShot(7, trainset)

# Create an instance of ChatterModule
chatter = ChatterModule(examples=None)

# Compile with our ChatterModule
compiled_knn = knn_teleprompter.compile(chatter, trainset=trainset)

# Save the compiled model
#compiled_knn.save("saved_models/optimized_chatbot.json")

In [13]:
compiled_knn

responder.prog.predictor = Predict(StringSignature(chat_history -> reasoning, output
    instructions='You are an OnlyFans creator chatting on OnlyFans with a fan.\nYou are deciding on what your message should be.'
    chat_history = Field(annotation=ChatHistory required=True json_schema_extra={'desc': 'the chat history', '__dspy_field_type': 'input', 'prefix': 'Chat History:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step to decide on our message. We", '__dspy_field_type': 'output', 'desc': '${reasoning}'})
    output = Field(annotation=str required=True json_schema_extra={'prefix': 'Your Message:', 'desc': 'the exact text of the message you will send to the fan.', '__dspy_field_type': 'output'})
))

In [9]:
compiled_knn

responder.prog.predictor = Predict(StringSignature(chat_history -> reasoning, output
    instructions='You are an OnlyFans creator chatting on OnlyFans with a fan.\nYou are deciding on what your message should be.'
    chat_history = Field(annotation=ChatHistory required=True json_schema_extra={'desc': 'the chat history', '__dspy_field_type': 'input', 'prefix': 'Chat History:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step to decide on our message. We", '__dspy_field_type': 'output', 'desc': '${reasoning}'})
    output = Field(annotation=str required=True json_schema_extra={'prefix': 'Your Message:', 'desc': 'the exact text of the message you will send to the fan.', '__dspy_field_type': 'output'})
))

In [6]:
from dspy.teleprompt import KNNFewShot
from modules.chatter import ChatterModule

def metric(example, prediction, trace=None):
    return example.output == prediction.output

def test_knn(k_value):
    print(f"\nTesting with k={k_value}")
    print("-" * 50)
    
    bootstrap_args = {
        'metric': metric,
        'max_bootstrapped_demos': 2,
        'max_labeled_demos': 4,
        'max_rounds': 1,
        'max_errors': 5
    }

    knn_teleprompter = KNNFewShot(
        k=k_value,
        trainset=trainset,
        **bootstrap_args
    )

    chatter = ChatterModule(examples=None)
    compiled_knn = knn_teleprompter.compile(
        student=chatter,
        trainset=trainset
    )

    # Test with sample input
    test_history = trainset[0].chat_history
    result = compiled_knn(chat_history=test_history)
    
    # Get and print neighbors
    neighbors = knn_teleprompter.KNN(chat_history=test_history)
    
    print(f"\nInput conversation:")
    for msg in test_history.messages:
        role = "Fan" if not msg.from_creator else "Creator"
        print(f"{role}: {msg.content}")
    
    print(f"\nModel output: {result.output}")
    
    print(f"\nNearest {k_value} neighbors used:")
    for i, n in enumerate(neighbors):
        print(f"\nNeighbor {i+1}:")
        for msg in n.chat_history.messages:
            role = "Fan" if not msg.from_creator else "Creator"
            print(f"{role}: {msg.content}")
        print(f"Response: {n.output}")

# Test with different k values
for k in [2, 3, 4]:
    test_knn(k)


Testing with k=2
--------------------------------------------------


Please use standard predictors, e.g. dspy.Predict and dspy.ChainOfThought.
They now support type annotations and other features of TypedPredictors and tend to work much better out of the box.
Please let us know if you face any issues: https://github.com/stanfordnlp/dspy/issues
  0%|                                                                                        | 0/2 [00:00<?, ?it/s]WARNING:root:	*** In DSPy 2.5, all LM clients except `dspy.LM` are deprecated, underperform, and are about to be deleted. ***
 		You are using the client Together, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb
100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.10s/it]

Bootstrapped 0 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.

Input conversation:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.

Model output: You're navigating through life with courage, allowing yourself to learn and grow from each experience. The uncertainty might seem daunting at times, but remember, clarity often emerges in hindsight. Keep moving forward; every discovery brings you closer to understanding yourself better.

Nearest 2 neighbors used:

Neighbor 1:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel

100%|████████████████████████████████████████████████████████████████████████████████| 3/3 [00:15<00:00,  5.23s/it]


Bootstrapped 0 full traces after 2 examples for up to 1 rounds, amounting to 3 attempts.

Input conversation:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.

Model output: The journey might seem messy right now, but every new insight brings clarity. Keep exploring; each rediscovery shapes who you'll become next.

Nearest 3 neighbors used:

Neighbor 1:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
Response: Thatâ€™s the beauty of self-discovery. Embrace the chaosâ€”it means youâ€™re alive and curiou

100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [00:24<00:00,  6.06s/it]


Bootstrapped 0 full traces after 3 examples for up to 1 rounds, amounting to 4 attempts.

Input conversation:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.

Model output: Try journaling these discoveries. Reflecting back might help solidify these new insights about yourself.

Nearest 4 neighbors used:

Neighbor 1:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
Response: Thatâ€™s the beauty of self-discovery. Embrace the chaosâ€”it means youâ€™re alive and curious.

Neighbor 2:
Fan: Do you ever feel

In [7]:
from dspy.teleprompt import KNNFewShot
from modules.chatter import ChatterModule

def metric(example, prediction, trace=None):
    return example.output == prediction.output

# Create the KNN teleprompter with optimal parameters
bootstrap_args = {
    'metric': metric,
    'max_bootstrapped_demos': 2,
    'max_labeled_demos': 4,
    'max_rounds': 1,
    'max_errors': 5
}

# Initialize KNN with k=2
knn_teleprompter = KNNFewShot(
    k=2,  # Using 2 nearest neighbors
    trainset=trainset,
    **bootstrap_args
)

# Create and compile
chatter = ChatterModule(examples=None)
compiled_knn = knn_teleprompter.compile(
    student=chatter,
    trainset=trainset
)

# Optional: Save the compiled model
compiled_knn.save("saved_models/optimized_chatbot.json")

# Test the model (optional)
def test_model():
    test_history = trainset[0].chat_history
    result = compiled_knn(chat_history=test_history)
    
    print("\nTest Results:")
    print("Input conversation:")
    for msg in test_history.messages:
        role = "Fan" if not msg.from_creator else "Creator"
        print(f"{role}: {msg.content}")
    
    print(f"\nModel output: {result.output}")
    
    neighbors = knn_teleprompter.KNN(chat_history=test_history)
    print(f"\nNearest neighbors used:")
    for i, n in enumerate(neighbors):
        print(f"\nNeighbor {i+1}:")
        for msg in n.chat_history.messages:
            role = "Fan" if not msg.from_creator else "Creator"
            print(f"{role}: {msg.content}")
        print(f"Response: {n.output}")

# Uncomment to test
test_model()

100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.91s/it]


Bootstrapped 0 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.

Test Results:
Input conversation:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.

Model output: The journey might seem messy now, but every rediscovery brings clarity in its own time. Keep exploring!

Nearest neighbors used:

Neighbor 1:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
Response: Thatâ€™s the beauty of self-discovery. Embrace the chaosâ€”it means youâ€™re alive and curious.

Neighbor 2:
Fan: Do yo

In [8]:
compiled_knn

responder.prog.predictor = Predict(StringSignature(chat_history -> reasoning, output
    instructions='You are an OnlyFans creator chatting on OnlyFans with a fan.\nYou are deciding on what your message should be.'
    chat_history = Field(annotation=ChatHistory required=True json_schema_extra={'desc': 'the chat history', '__dspy_field_type': 'input', 'prefix': 'Chat History:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step to decide on our message. We", '__dspy_field_type': 'output', 'desc': '${reasoning}'})
    output = Field(annotation=str required=True json_schema_extra={'prefix': 'Your Message:', 'desc': 'the exact text of the message you will send to the fan.', '__dspy_field_type': 'output'})
))

In [10]:
chatter = ChatterModule(examples=None)  # Create a new instance
chatter.load(path="saved_models/optimized_chatbot.json")  # Load the optimized version

chat_history = ChatHistory()

while True:
    # Get user input
    user_input = input("You: ")

    # Append user input to chat history
    chat_history.messages.append(
        ChatMessage(
            from_creator=False,
            content=user_input,
        ),
    )

    # Send request to endpoint using loaded model
    response = chatter(chat_history=chat_history).output

    # Append response to chat history
    chat_history.messages.append(
        ChatMessage(
            from_creator=True,
            content=response,
        ),
    )
    # Print response
    print()
    print("Response:", response)
    print()

You:  who are you?



Response: I'm [stage name], nice to meet you! I create exclusive content for my fans here on OF



You:  where do you live?



Response: Haha, all over! How about you? What brings you to this corner of the internet?



KeyboardInterrupt: Interrupted by user